In [4]:
#! /usr/bin/env python3
from astropy.table import Table, Column
from astropy.io.votable import from_table, writeto
from astropy.coordinates import SkyCoord, search_around_sky
import numpy as np
import astropy.units as u
import argparse
import numpy as np
from scipy.optimize import curve_fit
from astropy.io import fits
import logging 
import pandas as pd
import matplotlib.pyplot as plt 
import os
from scipy.stats import chi2
from matplotlib.ticker import ScalarFormatter
import read_prep_data
import pickle

In [110]:
def plot_sed(freq, flux, err_flux, fit_params, source_name, outpath="/data/gleam_x/drII_plotting/plots/", plot_extras = None, survey_legend = False):
    fig, ax = plt.subplots(1,1)

    fluxnegind = np.where((flux <= 0))[0]
    
    if len(fluxnegind) > 0:
        logger.debug(f"There are some negative fluxes???")
        fluxposind = np.where(flux > 0)[0]
        flux = flux[fluxposind]
        err_flux = err_flux[fluxposind]


    min_ind = np.where(flux == np.nanmin(flux))[0][0]
    max_ind = np.where(flux == np.nanmax(flux))[0][0]

    nu = np.geomspace(
        np.min(freq),
        1000,
        100
    )

    ax.errorbar(
        freq,
        flux,
        yerr=err_flux,
        ls='None',
        marker='.',
        color="hotpink"
    )
    if plot_extras is not None: 
        for nm in plot_extras.keys():
            extra_dict = plot_extras[nm]
            if survey_legend is True: 
                ax.errorbar(
                    extra_dict["freq"].to(u.MHz).value,
                    extra_dict["flux"].to(u.Jy).value,
                    yerr=extra_dict["err_flux"].to(u.Jy).value,
                    ls='None',
                    marker=extra_dict["marker"],
                    label=extra_dict["label"],
                    color='k',
                    alpha=0.5,
                )
            else: 
                ax.errorbar(
                    extra_dict["freq"].to(u.MHz).value,
                    extra_dict["flux"].to(u.Jy).value,
                    yerr=extra_dict["err_flux"].to(u.Jy).value,
                    ls='None',
                    marker=extra_dict["marker"],
                    color='k',
                    alpha=0.5,
                )
            np.append(flux, extra_dict["flux"].to(u.Jy).value)

    legend = False

    if fit_params is not None:
        legend = True
        for fits in fit_params.keys():
            fit_dict = fit_params[fits]
            ax.plot(
                nu,
                fit_dict["model"](nu, *fit_dict["best_params"]),
                alpha=0.2,
                color=fit_dict["colour"],
            )

    if legend is True:
        ax.legend()
    
    ax.loglog()
    ax.set(
        xlabel='Frequency (MHz)',
        ylabel='Integrated Flux (Jy)',
        title=f"{source_name.format('latex')}",
    )
    # TODO: NEED TO UPDATE WITH ALL EXTRAS
    y_min = (flux[min_ind]-err_flux[min_ind])*0.7
    if y_min < 0: 
        logger.debug(f"The ymin is less than zero, going to set limit to 0.8*flux")
        y_min = np.abs(flux[min_ind]*0.8) 
    y_max = (flux[max_ind]+err_flux[max_ind])*1.2

    ax.tick_params(
        axis="both", which="major", direction="in", length=majorticklength, width=tickwidth, pad=5
    )
    ax.tick_params(
        axis="both", which="minor", direction="in", length=minorticklength, width=tickwidth
    )

    ax.set_xticks(x_ticks)
    ax.set_ylim([y_min, y_max])
    
    ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    ax.xaxis.set_minor_formatter(ScalarFormatter(useMathText=True))
    ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    ax.yaxis.set_minor_formatter(ScalarFormatter(useMathText=True))
    
    savenm = str(source_name)
    savenm = savenm.replace(" ","_")
    fig.tight_layout()
    try: 
        fig.savefig(f"{outpath}/{savenm}.png")
    except FileNotFoundError:
        logger.debug(f"Couldnt save the sed because there's no folder: {outpath}, will just save here...")
        fig.savefig(f"./{savenm}.png")

    plt.close(fig)


In [7]:
data_dir = "/data/gleam_x/drII_plotting/data"
save_dir = "/data/gleam_x/drII_plotting/plots"

CHANSN_FINAL = [
    "076",
    "084",
    "092",
    "099",
    "107",
    "115",
    "122",
    "130",
    "143",
    "151",
    "158",
    "166",
    "174",
    "181",
    "189",
    "197",
    "204",
    "212",
    "220",
    "227",
    ]
GLEAMX_FREQS_FINAL = np.array([float(i) for i in CHANSN_FINAL])
GLEAMX_INT_FINAL = [f"int_flux_{c}" for c in CHANSN_FINAL]
GLEAMX_ERR_FINAL = [f"err_int_flux_{c}" for c in CHANSN_FINAL]
GLEAMX_RMS_FINAL = [f"local_rms_{c}" for c in CHANSN_FINAL]
extra_names = ["racs", "tgss", "nvss", "vlass", "vla", "parkes"]
colours = ["hotpink", "darkviolet", "mediumblue"]
extra_markers = ["*", "s", "p"]
x_ticks=[70, 100, 150, 200, 300, 500, 800, 1400, 3000, 5000, 10000]
majorticklength=2.5
minorticklength=2
tickwidth=0.5
racs_freq = 887.5
sumss_freq =843
nvss_freq = 1400 


In [41]:
poster_src_fluxes = {
    "racs19": {
        "flux": [467.423*u.mJy],
        "err_flux": [0.898*u.mJy],
        "freq":[887*u.MHz],
        "marker":"*",
        "label": "RACS",
        "obstime": 2020,
    },
    "racsmid": {
        "flux": [245.879*u.mJy],
        "err_flux": [0.340*u.mJy],
        "freq":[1367*u.MHz],
        "marker":"*",
        "label": "RACSmid",
        "obstime": 2020, 
    },
    "sumss": {
        "flux": [500.5*u.mJy],
        "err_flux": [15.1*u.mJy],
        "freq": [845*u.MHz],
        "marker": "x",
        "label": "SUMSS",
    },
    "mrc": {
        "flux": [0.98*u.Jy],
        "err_flux": [0.03*u.mJy],
        "freq": [408*u.MHz],
        "marker": "1",
        "label": "MRC",
    },
    "tgss": {
        "flux": [1207*u.mJy],
        "err_flux":[120.7*u.mJy],
        "freq": [150*u.MHz],
        "marker": "s",
        "label": "TGSS"
    },
    "gleamyr1" : {
        "flux": [1.2145169*u.Jy, 1.371842*u.Jy , 1.4334112*u.Jy, 1.4469645*u.Jy, 1.4322087*u.Jy, 1.4640739*u.Jy,1.4335005*u.Jy, 1.4224626*u.Jy, 1.4363129*u.Jy, 1.4302706*u.Jy, 1.3815901*u.Jy, 1.3777914*u.Jy, 1.3595188*u.Jy, 1.2697446*u.Jy, 1.3053327*u.Jy, 1.2160496*u.Jy],
       "err_flux": [0.05144753*u.Jy, 0.05070164*u.Jy, 0.05074232*u.Jy, 0.04989807*u.Jy, 0.04595367*u.Jy,
       0.04623701*u.Jy, 0.04494362*u.Jy, 0.04439759*u.Jy, 0.04490269*u.Jy, 0.04434995*u.Jy,
       0.04293044*u.Jy, 0.0431119*u.Jy , 0.0434337*u.Jy , 0.04106788*u.Jy, 0.04203401*u.Jy,
       0.03975713*u.Jy],
       "freq": [107.*u.MHz, 115.*u.MHz, 123.*u.MHz, 130.*u.MHz, 143.*u.MHz, 150.*u.MHz, 158.*u.MHz, 166.*u.MHz, 174.*u.MHz, 181.*u.MHz, 189.*u.MHz, 197.*u.MHz, 204.*u.MHz, 212.*u.MHz, 220.*u.MHz, 227.*u.MHz],
       "marker": ".",
       "label": "GLEAMyr1",
    },
    "gleamyr2" : {
        "flux": [1.587176*u.Jy , 1.7045547*u.Jy, 1.7525094*u.Jy, 1.8116142*u.Jy, 1.8089162*u.Jy, 1.81814*u.Jy  ,1.8211228*u.Jy, 1.7814379*u.Jy, 1.754858*u.Jy , 1.722707*u.Jy , 1.6615528*u.Jy, 1.6130741*u.Jy,1.560257*u.Jy , 1.5414249*u.Jy, 1.4970093*u.Jy, 1.4424244*u.Jy],
       "err_flux": [0.05618442*u.Jy, 0.0567744*u.Jy , 0.05676178*u.Jy, 0.05792917*u.Jy, 0.05616213*u.Jy, 0.05596262*u.Jy, 0.05573366*u.Jy, 0.0543611*u.Jy , 0.0534337*u.Jy , 0.05238493*u.Jy,0.0505539*u.Jy , 0.04905606*u.Jy, 0.04799765*u.Jy, 0.0475545*u.Jy , 0.0464403*u.Jy ,0.04524212*u.Jy],
       "freq": [107.*u.MHz, 115.*u.MHz, 123.*u.MHz, 130.*u.MHz, 143.*u.MHz, 150.*u.MHz, 158.*u.MHz, 166.*u.MHz, 174.*u.MHz, 181.*u.MHz, 189.*u.MHz,   197.*u.MHz, 204.*u.MHz, 212.*u.MHz, 220.*u.MHz, 227.*u.MHz],
       "marker": ".",
       "label": "GLEAMyr2",
    },
}
with open(f"{data_dir}/SED_fits/GLEAM-X_J223933.9-451417_extra_fluxes.pkl", "wb") as fp:
    pickle.dump(poster_src_fluxes, fp)

In [42]:
remanantsrc = {
    "atca": {
        "flux": [1*u.mJy],
        "err_flux": [0.898*u.mJy],
        "freq":[5.5*u.GHz],
        "marker":"*",
        "label": "ATCA",
        "obstime": 2020,
    },
}
with open(f"{data_dir}/SED_fits/GLEAM-X_J023558.0-442619_extra_fluxes.pkl", "wb") as fp:
    pickle.dump(remanantsrc, fp)

In [33]:
new_src_fluxes = {
    "racs20": {
        "flux": [570.734*u.mJy],
        "err_flux": [0.569*u.mJy],
        "freq":[887*u.MHz],
        "marker":"*",
        "label": "RACS20",
        "obstime": 2020,
    },
    "racs22" : {
        "flux": [610.275*u.mJy],
        "err_flux": [0.429*u.mJy],
        "freq": [887*u.MHz],
        "marker": "*",
        "label": "RACS22",
        "obstime": 2022,
    },
    "vlass": {
        "flux": [1012.8*u.mJy],
        "err_flux": [2.221*u.mJy],
        "freq":[ 3*u.GHz],
        "marker": "X",
        "label": "VLASS",
    },
    "at20g20": {
        "flux": [957*u.mJy],
        "err_flux": [48*u.mJy],
        "freq": [20*u.GHz],
        "marker": "x",
        "label": "AT20G",
    },
    "at20g8": {
        "flux": [772*u.mJy],
        "err_flux": [38*u.mJy],
        "freq": [8*u.GHz],
        "marker": "x",
        "label": "AT20G",
    },
    "at20g5": {
        "flux": [656*u.mJy],
        "err_flux": [33*u.mJy],
        "freq":[ 5*u.GHz],
        "marker": "x",
        "label": "AT20G",
    },
    "mrc": {
        "flux": [0.81*u.Jy],
        "err_flux": [0.06*u.mJy],
        "freq": [408*u.MHz],
        "marker": "1",
        "label": "MRC",
    },
    "nvss": {
        "flux": [782.1*u.mJy],
        "err_flux": [23.5*u.mJy],
        "freq": [1.4*u.GHz],
        "marker": "p",
        "label": "NVSS",
    },
    "tgss": {
        "flux": [378*u.mJy],
        "err_flux":[38.3*u.mJy],
        "freq": [150*u.MHz],
        "marker": "s",
        "label": "TGSS"
    },
    "gleamyr1" : {
        "flux": [0.5832954 *u.Jy, 0.5667906 *u.Jy, 0.54180264*u.Jy, 0.52235967*u.Jy, 0.5042521*u.Jy , 0.5336121*u.Jy , 0.53273016*u.Jy, 0.51573724*u.Jy, 0.51464796*u.Jy, 0.49480072*u.Jy, 0.509351*u.Jy  , 0.491719*u.Jy  , 0.48813605*u.Jy, 0.48157573*u.Jy, 0.46986744*u.Jy, 0.50347*u.Jy   ],
       "err_flux": [0.03139286*u.Jy, 0.0281583*u.Jy , 0.02622145*u.Jy, 0.02437806*u.Jy, 0.02187295*u.Jy, 0.02168303*u.Jy, 0.02095023*u.Jy, 0.02016683*u.Jy, 0.0208544*u.Jy , 0.02065154*u.Jy, 0.01957429*u.Jy, 0.01934798*u.Jy, 0.01850351*u.Jy, 0.01930788*u.Jy, 0.01777244*u.Jy, 0.01920473*u.Jy],
       "freq": [107.*u.MHz, 115.*u.MHz, 123.*u.MHz, 130.*u.MHz, 143.*u.MHz, 150.*u.MHz, 158.*u.MHz, 166.*u.MHz, 174.*u.MHz, 181.*u.MHz, 189.*u.MHz, 197.*u.MHz, 204.*u.MHz, 212.*u.MHz, 220.*u.MHz, 227.*u.MHz],
       "marker": ".",
       "label": "GLEAMyr1",
    },
    "gleamyr2" : {
        "flux": [0.5867444*u.Jy , 0.5646245*u.Jy , 0.5327771*u.Jy , 0.5219601*u.Jy , 0.5010619*u.Jy ,0.5006502*u.Jy , 0.5056668*u.Jy , 0.49456182*u.Jy, 0.48360512*u.Jy, 0.48060146*u.Jy, 0.47827613*u.Jy, 0.4830299*u.Jy , 0.46777195*u.Jy, 0.47250733*u.Jy, 0.4809157*u.Jy , 0.47253746*u.Jy],
       "err_flux": [0.0269183*u.Jy , 0.02406013*u.Jy, 0.02168781*u.Jy, 0.0208184*u.Jy , 0.01865202*u.Jy, 0.01820496*u.Jy, 0.01789274*u.Jy, 0.01725656*u.Jy, 0.0168783*u.Jy , 0.01647265*u.Jy, 0.01629787*u.Jy, 0.0163438*u.Jy , 0.01603665*u.Jy, 0.0161229*u.Jy , 0.01637955*u.Jy,       0.01627879*u.Jy],
       "freq": [107.*u.MHz, 115.*u.MHz, 123.*u.MHz, 130.*u.MHz, 143.*u.MHz, 150.*u.MHz, 158.*u.MHz, 166.*u.MHz, 174.*u.MHz, 181.*u.MHz, 189.*u.MHz,   197.*u.MHz, 204.*u.MHz, 212.*u.MHz, 220.*u.MHz, 227.*u.MHz],
       "marker": ".",
       "label": "GLEAMyr2",
    },
}


In [34]:
with open(f"{data_dir}/SED_fits/GLEAM-X_J233355.2-234340_extra_fluxes.pkl", "wb") as fp:
    pickle.dump(new_src_fluxes, fp)

In [29]:
# reading in RACS for xmatch (and others)
# racs_cat = Table.read(f"/data/gleam_x/PSS_analysis/data/RACS_DR1.vot").to_pandas()

# gonna just manually report the lit values: 
extra_fluxes = {
    "racs19": {
        "flux": [90.226*u.mJy],
        "err_flux": [0.454*u.mJy],
        "freq": [887.5*u.MHz],
        "marker": "*",
        "label": "RACS",
        "obstime": 2019,
    },
    "racss22": {
        "flux": [93.355*u.mJy],
        "err_flux": [0.316*u.mJy],
        "freq": [887.5*u.MHz],
        "marker": "*",
        "label": "RACS22",
        "obstime": 2022,
    },
    "tgss": {
        "flux": [65.1*u.mJy],
        "err_flux": [9.1*u.mJy],
        "freq": [150*u.MHz],
        "marker": "s",
        "label": "TGSS"
    },
    "nvss": {
        "flux": [78.2*u.mJy],
        "err_flux": [2.4*u.mJy],
        "freq": [1.4*u.GHz],
        "marker": "p",
        "label": "NVSS"
    },
    "vlass": {
        "flux": [71.211*u.mJy],
        "err_flux": [0.382*u.mJy],
        "freq": [3*u.GHz],
        "marker": "X",
        "label": "VLASS"
    },
    "vla": {
    "flux": [55.8*u.mJy],
        "err_flux": [2.79*u.mJy],
        "freq": [8.4*u.GHz],
        "marker": "D",
        "label": "VLA",
        "obstime": 2007,
    },
    "parkes": {
        "flux": [105*u.mJy],
        "err_flux": [12*u.mJy],
        "freq": [4.85*u.GHz],
        "marker": "<",
        "label": "Parkes",
        "obstime": 1994,
    },
    "vlass22": {
        "flux": [45*u.mJy],
        "err_flux": [2.3*u.mJy],
        "freq": [3*u.GHz],
        "marker": "+",
        "label": "VLASS22",
        "obstime": 2022,
    },
    "vlass19": {
        "flux": [59*u.mJy],
        "err_flux": [3*u.mJy],
        "freq": [3*u.GHz],
        "marker": "x",
        "label": "VLASS19",
        "obstime": 2019
    }
}


In [30]:
with open(f"{data_dir}/SED_fits/GLEAM-X_J235129.8-142756_extra_fluxes.pkl", "wb") as fp:
    pickle.dump(extra_fluxes, fp)

In [111]:
plot_sed(freq, flux, err_flux, source_fits, src_name, outpath="/data/gleam_x/drII_plotting/plots/", plot_extras = extra_fluxes)

legend:WARNING:1217 No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [3]:
main_cat = Table.read(f"{data_dir}/GLEAMX_DRII_filtered_prime_sedfit_paper_ucd.fits").to_pandas()
main_coords = SkyCoord(main_cat["RAJ2000"], main_cat["DEJ2000"],frame="fk5",unit=u.deg)


logger:WARNING:211 UnitsWarning: 'h:m:s' did not parse as fits unit: Invalid character at col 1 If this is meant to be a custom unit, define it with 'u.def_unit'. To have it recognized inside a file reader or other code, enable it with 'u.add_enabled_units'. For details, see https://docs.astropy.org/en/latest/units/combining_and_defining.html
logger:WARNING:211 UnitsWarning: 'd:m:s' did not parse as fits unit: Invalid character at col 1 If this is meant to be a custom unit, define it with 'u.def_unit'. To have it recognized inside a file reader or other code, enable it with 'u.add_enabled_units'. For details, see https://docs.astropy.org/en/latest/units/combining_and_defining.html
logger:WARNING:211 UnitsWarning: '%' did not parse as fits unit: At col 0, Unit '%' not supported by the FITS standard.  If this is meant to be a custom unit, define it with 'u.def_unit'. To have it recognized inside a file reader or other code, enable it with 'u.add_enabled_units'. For details, see https://d

In [47]:
# details of source of interest: 
src_name = "GLEAM-X J235129.8-142756"
src_row = main_cat.loc[main_cat.Name == src_name.encode()]
print(src_row)

                              Name  background_wide  local_rms_wide  \
20058  b'GLEAM-X J235129.8-142756'        -0.000194        0.001021   

               ra_str          dec_str     RAJ2000  err_RAJ2000    DEJ2000  \
20058  b'23:51:29.77'  b'-14:27:56.47'  357.874023     0.000064 -14.465687   

       err_DEJ2000  peak_flux_wide  ...  sp_alpha  err_sp_alpha  \
20058     0.000092        0.106829  ...       NaN           NaN   

       sp_reduced_chi2  csp_int_flux_fit_200  err_csp_int_flux_fit_200  \
20058              NaN              0.101567                  0.001096   

       csp_alpha  err_csp_alpha  csp_beta  err_csp_beta  csp_reduced_chi2  
20058   0.917768       0.085259  0.839486      0.132069          0.604577  

[1 rows x 388 columns]


In [48]:
freq, flux, err_flux = read_prep_data.get_freq_flux_err(src_row, apply_mask=False, internal_scale=0.02, freqs=GLEAMX_FREQS_FINAL, fluxes=GLEAMX_INT_FINAL, errs=GLEAMX_ERR_FINAL)

print(freq)
low_freq = freq[0:5]
low_flux = flux[0:5]
low_err_flux = err_flux[0:5]

high_freq = freq[5:]
high_flux = flux[5:]
high_err_flux = err_flux[5:]



[ 76.  84.  92.  99. 107. 115. 122. 130. 143. 151. 158. 166. 174. 181.
 189. 197. 204. 212. 220. 227.]


In [49]:
# Fitting powerlaws, maybe also something specific 
best_pl_low, chi2_pl_low, rchi2_pl_low = read_prep_data.fit_pl(low_freq, low_flux, low_err_flux)
best_pl_hig, chi2_pl_hig, rchi2_pl_hig = read_prep_data.fit_pl(high_freq, high_flux, high_err_flux)
best_cpl, chi2_cpl, rchi2_cpl = read_prep_data.fit_cpl(freq, flux, err_flux)


In [102]:
source_fits = {
    "pl_low": {
        "name": "PL",
        "model": read_prep_data.power_law,
        "best_params": best_pl_low,
        "chi2": chi2_pl_low,
        "rchi2": rchi2_pl_low,
        "colour": "hotpink",
    },
    "pl_high" : {
        "name": "PL",
        "model": read_prep_data.power_law,
        "best_params": best_pl_hig,
        "chi2": chi2_pl_hig,
        "rchi2": rchi2_pl_hig,
        "colour": "hotpink" 
    },
    "cpl_mwa": {
        "name": "CPL",
        "model": read_prep_data.curved_power_law,
        "best_params": best_cpl,
        "chi2": chi2_cpl,
        "rchi2": rchi2_cpl,
        "colour": "darkviolet"
    }
}

In [84]:
plot_sed(freq, flux, err_flux, source_fits, src_name, outpath="/data/gleam_x/drII_plotting/plots/", plot_extras = extra_fluxes)

0.04914184883236885
